## Data cleaning and processing

### Importing packages

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import spacy

### Downloading datasets

In [ ]:
# JD DATASET
df1 = pd.read_csv('data/job_title_des.csv')

print(df1.head())
print (df1['Job Title'].unique())

   Unnamed: 0             Job Title  \
0           0     Flutter Developer   
1           1      Django Developer   
2           2      Machine Learning   
3           3         iOS Developer   
4           4  Full Stack Developer   

                                     Job Description  
0  We are looking for hire experts flutter develo...  
1  PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...  
2  Data Scientist (Contractor)\n\nBangalore, IN\n...  
3  JOB DESCRIPTION:\n\nStrong framework outside o...  
4  job responsibility full stack engineer – react...  
['Flutter Developer' 'Django Developer' 'Machine Learning' 'iOS Developer'
 'Full Stack Developer' 'Java Developer' 'JavaScript Developer'
 'DevOps Engineer' 'Software Engineer' 'Database Administrator'
 'Wordpress Developer' 'PHP Developer' 'Backend Developer'
 'Network Administrator' 'Node js developer']


In [3]:
import pandas as pd

df2 = pd.read_csv('data/Preprocessed_Data.txt', sep=',')  # or sep=',' if comma-separated

print(df2.head())
print (df2['Category'].unique())

     Category                                               Text
0  Accountant  education omba executive leadership university...
1  Accountant  howard gerrard accountant deyjobcom birmingham...
2  Accountant  kevin frank senior accountant inforesumekraftc...
3  Accountant  place birth nationality olivia ogilvy accounta...
4  Accountant  stephen greet cpa senior accountant 9 year exp...
['Accountant' 'Advocate' 'Agriculture' 'Apparel' 'Architecture' 'Arts'
 'Automobile' 'Aviation' 'Banking' 'Blockchain' 'BPO'
 'Building and Construction' 'Business Analyst' 'Civil Engineer'
 'Consultant' 'Data Science' 'Database' 'Designing' 'DevOps'
 'Digital Media' 'DotNet Developer' 'Education' 'Electrical Engineering'
 'ETL Developer' 'Finance' 'Food and Beverages' 'Health and Fitness'
 'Human Resources' 'Information Technology' 'Java Developer' 'Management'
 'Mechanical Engineer' 'Network Security Engineer' 'Operations Manager'
 'PMO' 'Public Relations' 'Python Developer' 'React Developer' 'Sales'


In [4]:
df3 = pd.read_csv('data/UpdatedResumeDataSet.csv',encoding='utf-8')

print(df3.head())
print (df3['Category'].unique())

       Category                                             Resume
0  Data Science  Skills * Programming Languages: Python (pandas...
1  Data Science  Education Details \r\nMay 2013 to May 2017 B.E...
2  Data Science  Areas of Interest Deep Learning, Control Syste...
3  Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4  Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...
['Data Science' 'HR' 'Advocate' 'Arts' 'Web Designing'
 'Mechanical Engineer' 'Sales' 'Health and fitness' 'Civil Engineer'
 'Java Developer' 'Business Analyst' 'SAP Developer' 'Automation Testing'
 'Electrical Engineering' 'Operations Manager' 'Python Developer'
 'DevOps Engineer' 'Network Security Engineer' 'PMO' 'Database' 'Hadoop'
 'ETL Developer' 'DotNet Developer' 'Blockchain' 'Testing']


### Data Cleaning

In [14]:
nlp = spacy.load("en_core_web_sm")

def preprocess_resume(text):

    # Step 1: lowercase the entire text so spaCy works correctly
    text = text.lower()

    # Remove email addresses
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', ' ', text)

    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', ' ', text)

    # Remove phone numbers (all formats)
    text = re.sub(r'\+?\d[\d\-\s\(\)]{6,}\d', ' ', text)

    # Remove punctuation
    text = re.sub(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", ' ', text)

    # Remove non-ASCII chars
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # Remove numbers except 4-digit years
    text = re.sub(r'\b(?!\d{4}\b)\d+\b', ' ', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove PERSON names using spaCy (now more accurate because lowercase)
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            text = text.replace(ent.text, ' ')

    # Final cleanup
    text = re.sub(r'\s+', ' ', text).strip()

    return text


df2 = df2.rename(columns={
    'Text': 'Resume'
})
combined_df = pd.concat([df2, df3], ignore_index=True)

# Remove duplicate resumes and reset index
combined_df = combined_df.drop_duplicates(subset='Resume').reset_index(drop=True)

# Apply cleaning
combined_df['clean_text'] = combined_df['Resume'].apply(preprocess_resume)

print(combined_df.head())
print (combined_df['Category'].unique())

     Category                                             Resume  \
0  Accountant  education omba executive leadership university...   
1  Accountant  howard gerrard accountant deyjobcom birmingham...   
2  Accountant  kevin frank senior accountant inforesumekraftc...   
3  Accountant  place birth nationality olivia ogilvy accounta...   
4  Accountant  stephen greet cpa senior accountant 9 year exp...   

                                          clean_text  
0  education omba executive leadership university...  
1  accountant deyjobcom linkedinname work experie...  
2  senior accountant inforesumekraftcom chicago i...  
3  place birth nationality olivia ogilvy accounta...  
4  cpa senior accountant year experience establis...  
['Accountant' 'Advocate' 'Agriculture' 'Apparel' 'Architecture' 'Arts'
 'Automobile' 'Aviation' 'Banking' 'Blockchain' 'BPO'
 'Building and Construction' 'Business Analyst' 'Civil Engineer'
 'Consultant' 'Data Science' 'Database' 'Designing' 'DevOps'
 'Digital 

In [15]:
# Handling similar labels

# Health and Fitness & Health and fitness
# HR & Human Resources
# DevOps & DevOps Engineer
# Testing & Automation Testing
combined_df['Category'] = combined_df['Category'].replace('HR', 'Human Resources')
combined_df['Category'] = combined_df['Category'].replace('Health and fitness', 'Health and Fitness')
combined_df['Category'] = combined_df['Category'].replace('DevOps Engineer', 'DevOps')
combined_df['Category'] = combined_df['Category'].replace('Automation Testing', 'Testing')


# Remove Hadoop
# Business Process Outsourcing (BPO)
# Project Management Office (PMO) -> Project Management
combined_df = combined_df[combined_df['Category'] != 'Hadoop']
combined_df['Category'] = combined_df['Category'].replace('BPO', 'Business Process Outsourcing')
combined_df['Category'] = combined_df['Category'].replace('PMO', 'Project Management')

print(combined_df.head())
print (combined_df['Category'].unique())

     Category                                             Resume  \
0  Accountant  education omba executive leadership university...   
1  Accountant  howard gerrard accountant deyjobcom birmingham...   
2  Accountant  kevin frank senior accountant inforesumekraftc...   
3  Accountant  place birth nationality olivia ogilvy accounta...   
4  Accountant  stephen greet cpa senior accountant 9 year exp...   

                                          clean_text  
0  education omba executive leadership university...  
1  accountant deyjobcom linkedinname work experie...  
2  senior accountant inforesumekraftcom chicago i...  
3  place birth nationality olivia ogilvy accounta...  
4  cpa senior accountant year experience establis...  
['Accountant' 'Advocate' 'Agriculture' 'Apparel' 'Architecture' 'Arts'
 'Automobile' 'Aviation' 'Banking' 'Blockchain'
 'Business Process Outsourcing' 'Building and Construction'
 'Business Analyst' 'Civil Engineer' 'Consultant' 'Data Science'
 'Database' 'Desi

In [16]:
combined_df

,Category,Resume,clean_text
0,Accountant,education omba executive leadership university...,education omba executive leadership university...
1,Accountant,howard gerrard accountant deyjobcom birmingham...,accountant deyjobcom linkedinname work experie...
2,Accountant,kevin frank senior accountant inforesumekraftc...,senior accountant inforesumekraftcom chicago i...
3,Accountant,place birth nationality olivia ogilvy accounta...,place birth nationality olivia ogilvy accounta...
4,Accountant,stephen greet cpa senior accountant 9 year exp...,cpa senior accountant year experience establis...
...,...,...,...
12246,Testing,Computer Skills: â¢ Proficient in MS office (...,computer skills proficient in ms office word b...
12247,Testing,â Willingness to accept the challenges. â ...,willingness to accept the challenges positive ...
12248,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",personal skills quick learner eagerness to lea...
12249,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,computer skills software knowledge ms power po...


In [17]:
# Saving dataset
combined_df.to_csv('data/clean_resume_dataset.csv', index=False)
